In [1]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim
import os, re, csv, math, codecs
num_words=30000
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_words+2,)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

17473536/17464789 [==============================] - 0s 0us/step


In [2]:
num_words=len(np.unique(np.hstack(data)))
print("Categories:", np.unique(targets))
print("Number of unique words:", num_words)

Categories: [0 1]
Number of unique words: 30000


Agregar este archivo a la carpeta de google drive

https://drive.google.com/open?id=1jJ20oNZIysi-V-iORAE0eEZq4LhuWxUW

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 234.75892
Standard Deviation: 173


In [5]:
print("Label:", targets[0])

Label: 1
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[1]] )
print(decoded)

1654784/1641221 [==============================] - 0s 0us/step
# big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal measures the hair is big lots of boobs bounce men wear those cut tee shirts that show off their stomachs sickening that men actually wore them and the music is just # trash that plays over and over again in almost every scene there is trashy music boobs and paramedics taking away bodies and the gym still doesn't close for # all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh a

In [8]:
#load embeddings
EMBEDDING_DIR = "/content/drive/My Drive/"
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open(EMBEDDING_DIR+'glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...
found 400000 word vectors


In [9]:
embeddings_index["car"]

array([-0.1684  , -0.53827 ,  0.31155 , -0.53218 ,  0.26678 , -0.13638 ,
        0.36621 ,  0.68383 ,  0.77726 ,  0.68049 ,  0.69137 ,  0.2103  ,
        0.091065,  0.24845 , -0.16157 ,  0.46291 , -0.1503  ,  0.2562  ,
       -0.1199  ,  0.5913  ,  1.0351  , -0.2052  ,  0.30244 , -0.34101 ,
        0.6326  , -0.31603 , -0.9959  , -0.33583 ,  0.25161 ,  0.10323 ,
        0.019611,  0.54893 , -0.33433 ,  0.29617 ,  0.41218 ,  0.4207  ,
        0.25775 ,  0.12709 ,  0.80269 ,  0.61944 ,  0.54316 , -0.5941  ,
        0.87551 , -0.063686, -0.29117 ,  0.61609 ,  0.33376 ,  0.14488 ,
       -0.039021, -1.1849  , -0.45951 ,  0.15631 , -0.50818 ,  1.2357  ,
        0.30965 , -1.958   , -1.1872  ,  1.2027  ,  2.1138  ,  0.083629,
        0.54319 ,  0.78883 ,  0.35416 ,  0.87736 ,  0.54007 , -0.10454 ,
        0.075371, -0.45727 , -0.27466 ,  0.11838 , -0.49412 , -0.61325 ,
        0.071519, -0.57665 ,  0.21371 ,  0.62137 ,  1.4404  , -0.34033 ,
       -0.89958 , -0.69605 ,  0.74058 ,  0.52105 , 

In [10]:
embed_dim=100
embedding_matrix=np.zeros([num_words+4,embed_dim])
for word, idx in index.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx+3,:]=embeddings_index[word]

In [11]:
maxlen=1000

In [12]:
data = pad_sequences(data,maxlen=maxlen)

In [13]:
len(data[0])

1000

In [14]:
len(data[1])

1000

In [15]:
data=np.array(data)

In [16]:
data.shape

(50000, 1000)

In [18]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
nb_words=num_words+4
num_filters=64
model = Sequential()
model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters*2, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 100)         3000400   
                                                                 
 conv1d_2 (Conv1D)           (None, 1000, 64)          44864     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 500, 64)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 500, 128)          57472     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 128)              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [19]:
model.fit(data,targets,batch_size=32,epochs=20,validation_split=0.2)

Epoch 1/20
1250/1250 [==============================] - 36s 20ms/step - loss: 0.4770 - accuracy: 0.7564 - val_loss: 0.3908 - val_accuracy: 0.8212
Epoch 2/20
1250/1250 [==============================] - 24s 19ms/step - loss: 0.3472 - accuracy: 0.8487 - val_loss: 0.3520 - val_accuracy: 0.8444
Epoch 3/20
1250/1250 [==============================] - 24s 20ms/step - loss: 0.3107 - accuracy: 0.8669 - val_loss: 0.3181 - val_accuracy: 0.8597
Epoch 4/20
1250/1250 [==============================] - 24s 19ms/step - loss: 0.2852 - accuracy: 0.8807 - val_loss: 0.2925 - val_accuracy: 0.8724
Epoch 5/20
1250/1250 [==============================] - 24s 20ms/step - loss: 0.2597 - accuracy: 0.8927 - val_loss: 0.2883 - val_accuracy: 0.8755
Epoch 6/20
1250/1250 [==============================] - 24s 20ms/step - loss: 0.2438 - accuracy: 0.9000 - val_loss: 0.2944 - val_accuracy: 0.8755
Epoch 7/20
1250/1250 [==============================] - 25s 20ms/step - loss: 0.2211 - accuracy: 0.9101 - val_loss: 0.2921 -

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08007812,  0.10498047,  0.04980469, ...,  0.00366211,
         0.04760742, -0.06884766],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.13378906,  0.1640625 , -0.17382812, ..., -0.25585938,
        -0.11669922,  0.08984375],
       [-0.12353516,  0.21777344, -0.578125  , ...,  0.12695312,
         0.20507812, -0.00543213],
       [-0.2265625 , -0.05151367,  0.14941406, ..., -0.19238281,
         0.01190186, -0.06542969]])